In [8]:
import random
import time
import bisect
from datetime import datetime

#dicionario com os locais mais afetados historicamente pelas queimadas
locais = [
    {"nome": "Floresta Amazônica", "regiao": "Norte", "estado": "AM", "cidade": "Manaus", "area": 1000000},
    {"nome": "Pantanal",            "regiao": "Centro-Oeste", "estado": "MT", "cidade": "Cuiabá", "area": 500000},
    {"nome": "Cerrado",             "regiao": "Centro-Oeste", "estado": "GO", "cidade": "Goiânia", "area": 750000},
    {"nome": "Mata Atlântica",      "regiao": "Sudeste",      "estado": "SP", "cidade": "São Paulo", "area": 300000},
]

ocorrencias = {}
lista_protocolos = []
contador_protocolo = 1
relatorios_gerados = False

def gerar_protocolo():
    global contador_protocolo
    p = f"P{contador_protocolo:04d}"
    contador_protocolo += 1
    return p

def inserir_ocorrencia(dados):
    #insere novo relatório em dicionário e marca que existe relatórios 
    global relatorios_gerados
    protocolo = gerar_protocolo()
    ocorrencias[protocolo] = dados
    # bisect está sendo usado para manter a lista_protocolos sempre ordenada e p/ habilitar buscas binárias em O(log n)
    bisect.insort(lista_protocolos, protocolo)
    relatorios_gerados = True
    return protocolo

def busca_por_protocolo(protocolo):
    # busca o protocolo exato na lista ordenada com uso da busca binária
    idx = bisect.bisect_left(lista_protocolos, protocolo)
    if idx < len(lista_protocolos) and lista_protocolos[idx] == protocolo:
        return ocorrencias[protocolo]
    return None

def opcao_dados_sensoriais(usuario):
    registros = []
    alerta = None
    for local in locais:
        temp = random.uniform(20, 45)
        umid = random.uniform(10, 80)
        vento = random.uniform(0, 30)
        fumaca = random.choice(["nenhuma", "pouca", "moderada", "muita"])
        precip = random.uniform(0, 50)
        ndvi = random.uniform(0, 1)
        dados = {
            "local": local["nome"],
            "area": local["area"],
            "data": datetime.now().strftime("%Y-%m-%d"),
            "hora": datetime.now().strftime("%H:%M:%S"),
            "temperatura": round(temp, 1),
            "fumaca": fumaca,
            "umidade": round(umid, 1),
            "vento": round(vento, 1),
            "direcao_vento": random.choice(["N","S","L","O"]),
            "precipitacao": round(precip, 1),
            "ndvi": round(ndvi, 2),
            "usuario": usuario
        }
        registros.append(dados)
        # condição crítica que interrompe a coleta e dispara alerta para o cloud seeding
        if temp > 35 or umid < 30 or vento > 20:
            alerta = dados
            break
    if alerta is None:
        print("Condições climáticas normais. Nenhum risco de incêndio no momento")
        for r in registros:
            print(r)
        protocolo = inserir_ocorrencia({"tipo":"preventiva","registros":registros})
        print("Relatório gerado. Protocolo:", protocolo)
        return
    print(f"Alerta! Alteração nas condições climáticas no local {alerta['local']}. Possível risco de incêndio!")
    print(alerta)
    resposta = input("Ativar o Cloud Seeding? (Sim/Não): ").strip().lower()
    if resposta != "sim":
        justificativa = input("Justifique por que não: ")
        protocolo = inserir_ocorrencia({
            "tipo":"preventiva_cancelada",
            "local": alerta["local"],
            "justificativa": justificativa
        })
        print("Justificativa salva. Protocolo:", protocolo)
        print("Programa encerrado.")
        return
    print("Cloud Seeding ativado! Drones D2 a caminho!")
    time.sleep(5)  # simula tempo de voo até as nuvens
    print("Iniciando a dispersão de iodeto de prata nas nuvens-alvo!")
    time.sleep(5)
    print("Liberação finalizada com sucesso!")
    print("Ocorrência de chuvas detectadas no local!")
    print("Condições normalizadas. Drones D2 retornando à base")
    alerta.update({
        "temperatura": round(random.uniform(20, 30), 1),
        "umidade": round(random.uniform(50, 80), 1),
        "vento": round(random.uniform(0, 10), 1)
    })
    print(alerta)
    protocolo = inserir_ocorrencia({"tipo":"preventiva","registros":registros,"cloud_seeding":True})
    print("Relatório gerado. Protocolo:", protocolo)
    print("Status finalizado.")

def opcao_focos_incendio(usuario):
    ativos = random.sample(locais, random.randint(0, len(locais)))
    if not ativos:
        print("Sem ocorrências no momento. Volte mais tarde!")
        input("Pressione Enter para voltar")
        return
    print(f"{len(ativos)} focos de incêndio ativos")
    for i, loc in enumerate(ativos, 1):
        print(f"{i}) {loc['nome']} - {loc['regiao']}")
    escolha = int(input("Escolha o número do local prioritário: ").strip())
    local_escolhido = ativos[escolha - 1]
    print("Drones D2 com retardantes enviados para o local")
    nome_op = input("Nome da operação: ").strip()
    dados_op = {
        "tipo": "reativa",
        "nome_operacao": nome_op,
        "local": local_escolhido["nome"],
        "regiao": local_escolhido["regiao"],
        "estado": local_escolhido["estado"],
        "cidade": local_escolhido["cidade"],
        "area": local_escolhido["area"],
        "data": datetime.now().strftime("%Y-%m-%d"),
        "hora": datetime.now().strftime("%H:%M:%S"),
        "tempo_deslocamento": f"{random.randint(1,5)}h",
        "coordenadas": f"{round(random.uniform(-10,-2),2)}, {round(random.uniform(-60,-50),2)}",
        "responsavel": input("Responsável pela operação: ").strip(),
        "nome_equipe": input("Nome da equipe: ").strip(),
        "qtd_integrantes": int(input("Quantidade de integrantes: ").strip()),
        "integrantes": [
            input(f"Integrante {j+1}: ").strip()
            for j in range(int(input("Quantos integrantes listar? ").strip()))
        ]
    }
    protocolo = inserir_ocorrencia(dados_op)
    print("Status: Em andamento | Imagens capturadas")
    print("Relatório:", dados_op, "Protocolo:", protocolo)
    time.sleep(5)
    print("Equipe de terra no local prioritário")
    time.sleep(5)
    print("Fogo controlado. Missão cumprida! Drones retornando.")
    print("Status: Finalizado")

def opcao_buscar_ocorrencia():
    if not relatorios_gerados:  # só permite a busca se ja foi gerado pelo menos um relatório
        print("Nenhum relatório gerado. Vá para as opções 1 ou 2.")
        return
    termo = input("Pesquisar por protocolo, nome, região, estado, cidade ou tipo: ").strip().lower()
    res = busca_por_protocolo(termo.upper())
    if res:
        print("Encontrado por protocolo:", res)
        return
    encontrados = False
    for p, d in ocorrencias.items():
        campos = [
            d.get("nome_operacao",""),
            d.get("regiao",""),
            d.get("estado",""),
            d.get("cidade",""),
            d.get("tipo","")
        ]
        if any(termo in str(c).lower() for c in campos):
            print("Encontrado:", p, d)
            encontrados = True
    if not encontrados:
        print("Nenhuma ocorrência encontrada")

def main():
    usuario = input("Digite seu nome de usuário: ").strip()
    while True:  # repeteo menu até o usuário escolher sair
        print("\nBem-vindo ao Sistema \"NimbusTech D2\"")
        print("1) Verificar os dados sensoriais")
        print("2) Verificar focos de incêndio ativos")
        print("3) Buscar ocorrências")
        print("4) Sair")
        opc = input("Escolha uma opção: ").strip()
        if opc == "1":
            opcao_dados_sensoriais(usuario)
        elif opc == "2":
            opcao_focos_incendio(usuario)
        elif opc == "3":
            opcao_buscar_ocorrencia()
        elif opc == "4":
            break
        else:
            print("Opção inválida")

main()



Bem-vindo ao Sistema "NimbusTech D2"
1) Verificar os dados sensoriais
2) Verificar focos de incêndio ativos
3) Buscar ocorrências
4) Sair
Nenhum relatório gerado. Vá para as opções 1 ou 2.

Bem-vindo ao Sistema "NimbusTech D2"
1) Verificar os dados sensoriais
2) Verificar focos de incêndio ativos
3) Buscar ocorrências
4) Sair
Sem ocorrências no momento. Volte mais tarde!

Bem-vindo ao Sistema "NimbusTech D2"
1) Verificar os dados sensoriais
2) Verificar focos de incêndio ativos
3) Buscar ocorrências
4) Sair
Alerta! Alteração nas condições climáticas no local Floresta Amazônica. Possível risco de incêndio!
{'local': 'Floresta Amazônica', 'area': 1000000, 'data': '2025-06-06', 'hora': '20:38:10', 'temperatura': 44.7, 'fumaca': 'nenhuma', 'umidade': 29.5, 'vento': 28.4, 'direcao_vento': 'N', 'precipitacao': 31.9, 'ndvi': 0.21, 'usuario': 'John'}
Cloud Seeding ativado! Drones D2 a caminho!
Iniciando a dispersão de iodeto de prata nas nuvens-alvo!
Liberação finalizada com sucesso!
Ocorrênc